Import libraries and generate token


In [ ]:
%pip install --upgrade geopandas
%pip install geopy


In [ ]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import geopy
import geopandas as gpd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.point import Point




headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = {
    'full_name': 'Fabiano Moreira Alves',
    'email': 'fabianomalves@proton.me',
}

access_token = requests.post('https://begrowth.deta.dev/user/', headers=headers, json=json_data)
access_token_json = access_token.json()

print(access_token_json)




Split the user and key as dictionary to get the acess token

In [ ]:
# split dictionary into keys and values
keys = []
values = []
items = access_token_json.items()
for item in items:
    keys.append(item[0]), values.append(item[1])
 
# printing keys and values separately
string_acess_token = str(values[-1])
print(string_acess_token)







Consume the API, calling the endpoint https://begrowth.deta.dev/token=access_token, concatenatating the url with the token. Then, create a dataframe for the jason and print data frame result.

In [ ]:

url_dev = "https://begrowth.deta.dev/token="
url_dev_with_token = url_dev + string_acess_token
print(url_dev_with_token)

# pd.json_normalize(pd.json_normalize.to_dict(url_dev_with_token["results"]).explode("key2.subkey20").to_dict(orient="records"))


df_01 = pd.read_json(url_dev_with_token)

print(df_01.head())



Info about the data frame

In [ ]:
df_01.info()


Find duplicate rows and filtering them

In [ ]:
duplicate_rows = df_01[df_01.duplicated(['id', ])]
print(duplicate_rows)


In [ ]:
#df2 = pd.json_normalize(duplicate_rows['address'])
#print(df2)

In [ ]:
data = json.loads(requests.get(url_dev_with_token).text)
data_normalize = pd.json_normalize(data)
print(data_normalize)


In [ ]:
duplicate_rows = data_normalize[data_normalize.duplicated(['id', ])]
print(duplicate_rows)

Doing the reverse geocoding and creating address field for the singles rows

In [ ]:
geolocator = Nominatim(user_agent="test")


def reverse_geocoding(latitude, longitude):
    try:
        location = geolocator.reverse(Point(latitude, longitude))
        return location.raw['display_name']
    except:
        return None

duplicate_rows['address_state'] = np.vectorize(reverse_geocoding)(duplicate_rows['address.geo_latitude'], duplicate_rows['address.geo_longitude'])

print(duplicate_rows)